# GET información de los partidos

In [1]:
import requests
import json 
import os

headers = {
    "X-RapidAPI-Key": "ff1445247amshc1e1716dd1c0de1p1a1f0ejsnd1bcc39c4c98",
    "X-RapidAPI-Host": "sofascore.p.rapidapi.com"
}

### Obtenemos id de cada equipo en la Api de sofaescore

In [2]:
url = "https://sofascore.p.rapidapi.com/teams/search"

equipos = ["Real Madrid", "Barcelona", "Atlético de Madrid", "Valencia", "Sevilla", "Villarreal", "Real Sociedad", "Real Betis", "Athletic Club", "Celta Vigo", "Almeria", "Getafe", "Mallorca", "Girona", "Granada", "Cadiz", "Alavés", "Rayo Vallecano", "Osasuna", "Las Palmas"]

ids_equipos = {}

for equipo in equipos:
    response = requests.get(url, headers=headers, params={"name": equipo})
    data = response.json()
    equipo_id = data['teams'][0]['id']
    ids_equipos[equipo] = equipo_id
    print(f"{equipo}: {equipo_id}")
print("Equipos IDs:", ids_equipos)

Real Madrid: 2829
Barcelona: 2817
Atlético de Madrid: 77879
Valencia: 2828
Sevilla: 2833
Villarreal: 2819
Real Sociedad: 2824
Real Betis: 2816
Athletic Club: 2825
Celta Vigo: 2821
Almeria: 2858
Getafe: 2859
Mallorca: 2826
Girona: 24264
Granada: 33779
Cadiz: 4488
Alavés: 2885
Rayo Vallecano: 2818
Osasuna: 2820
Las Palmas: 6577
Equipos IDs: {'Real Madrid': 2829, 'Barcelona': 2817, 'Atlético de Madrid': 77879, 'Valencia': 2828, 'Sevilla': 2833, 'Villarreal': 2819, 'Real Sociedad': 2824, 'Real Betis': 2816, 'Athletic Club': 2825, 'Celta Vigo': 2821, 'Almeria': 2858, 'Getafe': 2859, 'Mallorca': 2826, 'Girona': 24264, 'Granada': 33779, 'Cadiz': 4488, 'Alavés': 2885, 'Rayo Vallecano': 2818, 'Osasuna': 2820, 'Las Palmas': 6577}


In [13]:
ids_equipos = [2829, 2817, 77879, 2828, 2833, 2819, 2824, 2816, 2825, 2821, 2858, 2859, 2826, 24264, 33779, 4488, 2885, 2818, 2820, 6577]
#ids_equipos = [2829, 2817,77879,2828] 

### Obtenemos ultimos partidos de los equipos

In [14]:
import requests

url = "https://sofascore.p.rapidapi.com/teams/get-last-matches"
equipos_data = {}

for team_id in ids_equipos:
    querystring = {"teamId": str(team_id), "pageIndex": "0"}

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()

        # Almacena la respuesta en el diccionario usando el ID del equipo como clave
        equipos_data[str(team_id)] = data
        print(f"GET datos {team_id}")
    else:
        print(f"Error al obtener datos para el equipo con ID {team_id}. Código de estado: {response.status_code}")

# Guarda el diccionario en un archivo JSON
file_name = 'estadisticas_partidos/estadisticas _ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    json.dump(equipos_data, file, indent=4)

print(f"Todos los datos guardados en{file_name}")

GET datos 2829
GET datos 2817
GET datos 77879
GET datos 2828
GET datos 2833
GET datos 2819
GET datos 2824
GET datos 2816
GET datos 2825
GET datos 2821
GET datos 2858
GET datos 2859
GET datos 2826
GET datos 24264
GET datos 33779
GET datos 4488
GET datos 2885
GET datos 2818
GET datos 2820
GET datos 6577
Todos los datos guardados enestadisticas_partidos/equipos_data.json
